In [1]:
!python -m pip install html2text

In [2]:
!python -m pip install mistletoe

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import json
import sys
import re
import logging
import pandas as pd
import requests
import warcio
import mistletoe

from warcio import ArchiveIterator
from contextlib import closing
from html2text import HTML2Text
from bs4 import BeautifulSoup
from pathlib import Path
from IPython.display import HTML as HTML_raw, display

In [5]:
def HTML(text):
    text = text.replace('$', r'\$')
    return HTML_raw(text)

In [6]:
def html2md(html):
    parser = HTML2Text()
    parser.ignore_images = True
    parser.ignore_anchors = True
    parser.body_width = 0
    md = parser.handle(html)
    return md

In [7]:
def html2plain(html):
    # HTML to Markdown
    md = html2md(html)
    # Normalise custom lists
    md = re.sub(r'(^|\n) ? ? ?\\?[•·–-—-*]( \w)', r'\1  *\2', md)
    # Convert back into HTML
    html_simple = mistletoe.markdown(md)
    # Convert to plain text
    soup = BeautifulSoup(html_simple)
    text = soup.getText()
    # Strip off table formatting
    text = re.sub(r'(^|\n)\|\s*', r'\1', text)
    # Strip off extra emphasis
    text = re.sub(r'\*\*', '', text)
    # Remove trailing whitespace and leading newlines
    text = re.sub(r' *$', '', text)
    text = re.sub(r'\n\n+', r'\n\n', text)
    text = re.sub(r'^\n+', '', text)
    return text

In [8]:
websites = pd.read_csv("./ccmain/ccmain_2021-04_Long2.txt", header = None, sep = '|') 

In [9]:
websites.head()

,0,1,2,3
0,NaN,https://globalannal.wordpress.com/tag/careers/...,crawl-data/CC-MAIN-2021-04/segments/1610703528...,NaN
1,NaN,https://globalannal.wordpress.com/tag/jobs-num...,crawl-data/CC-MAIN-2021-04/segments/1610703528...,NaN
2,NaN,https://globalannal.wordpress.com/tag/jobs-rep...,crawl-data/CC-MAIN-2021-04/segments/1610703528...,NaN
3,NaN,https://globalshapersbucharest.wordpress.com/2...,crawl-data/CC-MAIN-2021-04/segments/1610703506...,NaN
4,NaN,https://globalyouthbook.wordpress.com/tag/care...,crawl-data/CC-MAIN-2021-04/segments/1610704798...,NaN


In [10]:
websites.columns

Int64Index([0, 1, 2, 3], dtype='int64')

In [11]:
websites.drop([0, 3], axis=1, inplace=True)
websites.head()

,1,2
0,https://globalannal.wordpress.com/tag/careers/...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
1,https://globalannal.wordpress.com/tag/jobs-num...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
2,https://globalannal.wordpress.com/tag/jobs-rep...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
3,https://globalshapersbucharest.wordpress.com/2...,crawl-data/CC-MAIN-2021-04/segments/1610703506...
4,https://globalyouthbook.wordpress.com/tag/care...,crawl-data/CC-MAIN-2021-04/segments/1610704798...


In [12]:
websites.columns = ['url_link', 'warc_file']
websites.head()

,url_link,warc_file
0,https://globalannal.wordpress.com/tag/careers/...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
1,https://globalannal.wordpress.com/tag/jobs-num...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
2,https://globalannal.wordpress.com/tag/jobs-rep...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
3,https://globalshapersbucharest.wordpress.com/2...,crawl-data/CC-MAIN-2021-04/segments/1610703506...
4,https://globalyouthbook.wordpress.com/tag/care...,crawl-data/CC-MAIN-2021-04/segments/1610704798...


In [13]:
websites.url_link.value_counts()

https://seeker.worksourcewa.com/jobsearch/powersearch.aspx?where=Washington                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      7
https://www.xn--itbaw-ova.de/jobs/151816/werkstudent-w-strich-m-strich-d-fuer-bachelor-master-oder-projektarbeiten-softwareentwicklung-java-in-stuttgart-und-karlsruhe                                                               

In [14]:
websites.url_link.value_counts().keys()

Index(['https://seeker.worksourcewa.com/jobsearch/powersearch.aspx?where=Washington                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  ',
       'https://www.xn--itbaw-ova.de/jobs/151816/werkstudent-w-strich-m-strich-d-fuer-bachelor-master-oder-projektarbeiten-softwareentwicklung-java-in-stuttgart-und-karlsruhe                                                  

In [15]:
urlList = []
for i in websites.url_link.value_counts().keys():
    urlList.append(i.replace(' ',''))
    
print(urlList[:10])

['https://seeker.worksourcewa.com/jobsearch/powersearch.aspx?where=Washington', 'https://www.xn--itbaw-ova.de/jobs/151816/werkstudent-w-strich-m-strich-d-fuer-bachelor-master-oder-projektarbeiten-softwareentwicklung-java-in-stuttgart-und-karlsruhe', 'https://www.xn--itbaw-ova.de/jobs/152230/senior-software-engineer-w-strich-m-strich-d-sap-in-stuttgart-karlsruhe-und-reutlingen', 'https://www.xn--itbaw-ova.de/jobs/151789/werkstudent-w-strich-m-strich-d-softwareentwicklung-java-in-stuttgart-und-karlsruhe', 'https://www.xn--itbaw-ova.de/jobs/152501/consultant-w-strich-m-strich-d-sap-abap-in-stuttgart-karlsruhe-und-reutlingen', 'https://www.xn--itbaw-ova.de/jobs/152638/software-architect-w-strich-m-strich-d-java-in-stuttgart-karlsruhe-und-reutlingen', 'https://www.xn--itbaw-ova.de/jobs/151779/trainee-w-strich-m-strich-d-softwareentwicklung-java-in-stuttgart-karlsruhe-und-reutlingen', 'https://www.xn--itbaw-ova.de/jobs/152535/consultant-w-strich-m-strich-d-microsoft-business-intelligence-in-

In [16]:
!python -m pip install cdx_toolkit

In [17]:
import cdx_toolkit

In [18]:
cdx = cdx_toolkit.CDXFetcher(source='cc')

In [19]:
cdx_indexes = requests.get('https://index.commoncrawl.org/collinfo.json').json()

In [20]:
pd.options.display.max_colwidth=150
pd.options.display.max_rows=6

In [21]:
pd.DataFrame(cdx_indexes)

,id,name,timegate,cdx-api
0,CC-MAIN-2021-04,January 2021 Index,https://index.commoncrawl.org/CC-MAIN-2021-04/,https://index.commoncrawl.org/CC-MAIN-2021-04-index
1,CC-MAIN-2020-50,November 2020 Index,https://index.commoncrawl.org/CC-MAIN-2020-50/,https://index.commoncrawl.org/CC-MAIN-2020-50-index
2,CC-MAIN-2020-45,October 2020 Index,https://index.commoncrawl.org/CC-MAIN-2020-45/,https://index.commoncrawl.org/CC-MAIN-2020-45-index
...,...,...,...,...
75,CC-MAIN-2012,Index of 2012 ARC files,https://index.commoncrawl.org/CC-MAIN-2012/,https://index.commoncrawl.org/CC-MAIN-2012-index
76,CC-MAIN-2009-2010,Index of 2009 - 2010 ARC files,https://index.commoncrawl.org/CC-MAIN-2009-2010/,https://index.commoncrawl.org/CC-MAIN-2009-2010-index
77,CC-MAIN-2008-2009,Index of 2008 - 2009 ARC files,https://index.commoncrawl.org/CC-MAIN-2008-2009/,https://index.commoncrawl.org/CC-MAIN-2008-2009-index


In [22]:
print(pd.DataFrame(cdx_indexes).head(20).to_markdown())

|    | id              | name                 | timegate                                       | cdx-api                                             |
|---:|:----------------|:---------------------|:-----------------------------------------------|:----------------------------------------------------|
|  0 | CC-MAIN-2021-04 | January 2021 Index   | https://index.commoncrawl.org/CC-MAIN-2021-04/ | https://index.commoncrawl.org/CC-MAIN-2021-04-index |
|  1 | CC-MAIN-2020-50 | November 2020 Index  | https://index.commoncrawl.org/CC-MAIN-2020-50/ | https://index.commoncrawl.org/CC-MAIN-2020-50-index |
|  2 | CC-MAIN-2020-45 | October 2020 Index   | https://index.commoncrawl.org/CC-MAIN-2020-45/ | https://index.commoncrawl.org/CC-MAIN-2020-45-index |
|  3 | CC-MAIN-2020-40 | September 2020 Index | https://index.commoncrawl.org/CC-MAIN-2020-40/ | https://index.commoncrawl.org/CC-MAIN-2020-40-index |
|  4 | CC-MAIN-2020-34 | August 2020 Index    | https://index.commoncrawl.org/CC-MAIN-2020-34/

In [23]:
print(pd.DataFrame(cdx_indexes).tail(20).to_markdown())

|    | id                | name                           | timegate                                         | cdx-api                                               |
|---:|:------------------|:-------------------------------|:-------------------------------------------------|:------------------------------------------------------|
| 58 | CC-MAIN-2015-32   | July 2015 Index                | https://index.commoncrawl.org/CC-MAIN-2015-32/   | https://index.commoncrawl.org/CC-MAIN-2015-32-index   |
| 59 | CC-MAIN-2015-27   | June 2015 Index                | https://index.commoncrawl.org/CC-MAIN-2015-27/   | https://index.commoncrawl.org/CC-MAIN-2015-27-index   |
| 60 | CC-MAIN-2015-22   | May 2015 Index                 | https://index.commoncrawl.org/CC-MAIN-2015-22/   | https://index.commoncrawl.org/CC-MAIN-2015-22-index   |
| 61 | CC-MAIN-2015-18   | April 2015 Index               | https://index.commoncrawl.org/CC-MAIN-2015-18/   | https://index.commoncrawl.org/CC-MAIN-2015-18-index   

In [24]:
api_url = cdx_indexes[0]['cdx-api']
api_url

'https://index.commoncrawl.org/CC-MAIN-2021-04-index'

In [25]:
import zlib

In [26]:
count = 0
for url_link in urlList:
    count += 1
    r1 = requests.get(api_url,
                      params = {
                          'url': url_link,
                          'output': 'json',
                          'filter': ['=status:200', '=mime-detected:text/html']
                      }
                     )
    records = [json.loads(line) for line in r1.text.split('\n') if line]
    record = records[0]
    data_url = 'https://commoncrawl.s3.amazonaws.com/' + record['filename']
    headers = {'Range': f'bytes={int(record["offset"])}-{int(record["offset"]) + int(record["length"])}'}
    r2 = requests.get(data_url, headers=headers)
    data = zlib.decompress(r2.content, wbits = zlib.MAX_WBITS | 16)
    print(BeautifulSoup(data.decode('utf-8', errors='ignore'), 'html.parser').getText())
    if count == 1:
        break

WARC/1.0
WARC-Type: response
WARC-Date: 2021-01-18T13:09:11Z
WARC-Record-ID: 
Content-Length: 166986
Content-Type: application/http; msgtype=response
WARC-Warcinfo-ID: 
WARC-Concurrent-To: 
WARC-IP-Address: 208.71.193.225
WARC-Target-URI: https://seeker.worksourcewa.com/jobsearch/powersearch.aspx?where=Washington
WARC-Payload-Digest: sha1:G5TDJ5Y34ILZUXLSHBKOYTIE7OKZ4LWU
WARC-Block-Digest: sha1:DLKD7ZRNF2JPK7QHQNGMB7OL44HVQN4J
WARC-Identified-Payload-Type: text/html

HTTP/1.1 200 OK
Cache-Control: no-cache
Pragma: no-cache
Content-Type: text/html; charset=utf-8
Expires: -1
X-Frame-Options: SAMEORIGIN
Set-Cookie: DOMAINALIAS=; path=/
Set-Cookie: DOMAINALIAS=; expires=Sun, 17-Jan-2021 13:09:10 GMT; path=/
Set-Cookie: DOMAINALIAS=; path=/
Set-Cookie: DOMAINALIAS=; expires=Sun, 17-Jan-2021 13:09:10 GMT; path=/
P3P: CP="NOI ADM DEV PSAi COM NAV OUR OTR STP IND DEM"
Set-Cookie: DOMAINALIAS=; path=/
Set-Cookie: DOMAINALIAS=; expires=Sun, 17-Jan-2021 13:09:10 GMT; path=/
Set-Cookie: jslocation

In [ ]:
count = 0
for url_link in urlList:
    count += 1
    objs = list(cdx.iter(url_link, from_ts='202101', to='202102', output='json', 
                         filter=['=status:200', '=languages:eng', '=charset:UTF-8', '=mime-detected:text/html']))
    df = pd.DataFrame([o.data for o in objs])
    #print(df.shape)
    if (df.shape[0] == 0): continue
    html = objs[0].content.decode('utf-8', errors='ignore')  
    try:
        filename = BeautifulSoup(html, 'html.parser').head.title.text.replace('\t','').replace('\r\n','')
        f = open(str(count)+"_2021JAN_jobAD.txt","w", encoding='utf-8')
        print(count,": ",filename,' ',objs[0].warc_record.rec_headers.get_header('WARC-Target-URI'))
        md = html2md(html)
        html2 = mistletoe.markdown(md)
        soup = BeautifulSoup(html2, 'lxml')
        #print(soup.get_Text(' '))
        f.write(soup.get_text(' '))
        #html = "".join(line.strip() for line in html.split("\n"))
    except Exception as e:
        print(e)
    finally:
        #if count == 1:
        #break
        f.close()

1 :  WorkSourceWA   https://seeker.worksourcewa.com/jobsearch/powersearch.aspx?where=Washington
11 :  Can I use Google Analytics on my careers page? – Workable Support   https://help.workable.com/hc/en-us/articles/360053410833-Can-I-use-Google-Analytics-on-my-careers-page-
12 :  Career Tools & Advice | Careers New Zealand | Working In New Zealand   https://www.workingin-newzealand.com/career-tools-advice/
21 :  5 Ways To Avoid Frustration During A Long Job Search - Work It Daily | Where Careers Go To Grow   https://www.workitdaily.com/avoid-frustration-long-job-search
22 :  Which job boards can I post to through Workable? – Workable Support   https://help.workable.com/hc/en-us/articles/115012441868-Which-job-boards-can-I-post-to-through-Workable-
23 :  'Should I Quit My Job?' – 11 Critical Questions To Ask Yourself - Work It Daily | Where Careers Go To Grow   https://www.workitdaily.com/should-quit-job-questions
27 :  5 Easy Ways To Make Yourself A More Attractive Job Candidate - Work 

retrying after 1s for 503
retrying after 1s for 503


45 :  The Power Of Persistence In Your Career - Work It Daily | Where Careers Go To Grow   https://www.workitdaily.com/power-persistence-career
49 :  jobbers – Welcome to the Thunderdome   https://thunderdomedotblog.wordpress.com/tag/jobbers/
53 :  Cristiano Ronaldo Scores 700 Career goal – Halif's blog   https://halifsblog.wordpress.com/2019/10/14/cristiano-ronaldo-scores-700-career-goal/
54 :  Career – Jacqui in New Zealand   https://jacquiwucom.wordpress.com/tag/career/
55 :  The Tech sector will lead in rapid growth of jobs in every economy – The New Economy –  The Perfect Economy   https://theneweconomysg.wordpress.com/2020/06/10/the-tech-sector-will-lead-in-rapid-growth-of-jobs-in-every-economy/
56 :  What That Job Posting REALLY Means - Work It Daily | Where Careers Go To Grow   https://www.workitdaily.com/job-posting-translations
62 :  Job Boards Perfect for Diversity Recruiting Efforts - Workology   https://workology.com/18-job-boards-perfect-for-diversity-recruiting-complianc

retrying after 1s for 503


164 :  How do I post a job in multiple languages? – Workable Support   https://help.workable.com/hc/en-us/articles/360042905674-How-do-I-post-a-job-in-multiple-languages-
168 :  Niagara County Employment and Training - Worksource One > Job Seeker   http://worksource1.com/job-seeker
169 :  Vancouver technical writing jobs – Vancouver Writing Jobs   https://vancouverwritingjobs.wordpress.com/tag/vancouver-technical-writing-jobs/


...

6484 :  Next Level Jobs: An Opportunity to Earn a High-Value Certificate FOR FREE – Ivy Tech Evansville News   https://ivytechsouthwest.wordpress.com/2019/05/29/next-level-jobs-an-opportunity-to-earn-a-high-value-certificate-for-free/
6485 :  Finance Systems Data Analyst  jobs - Gleeson Group   https://www.workwithglee.com/job/finance-systems-data-analyst/
6487 :  Admin Support: Freelance Jobs Online - Workana   https://www.workana.com/jobs?category=admin-support&page=2
6489 :  
      Care Assistant 484857 |
     work4a.com
     http://www.work4a.com/search/jobs/484857/Care-Assistant.html
6490 :  WorkSourceWA - occda   https://www.worksourcewa.com/jobsearch/PowerSearch.aspx?q=occda&amp;rad_units=miles&amp;pp=25&amp;vw=b&amp;setype=2&amp;rad=20&amp;tm=0&sort=dt.rv.di&rad=50&tm=3
6492 :  ClojureScript / Frontend Developer in Austin - Skipp | Functional Works   https://functional.works-hub.com/jobs/clojurescript-slash-frontend-developer-787
6499 :  WorkSourceWA - occda   https://www.works

6600 :  Jobs – Word Stanza   https://wordstanza.com/category/jobs/
6603 :  4 Steps To Properly Quit Your Job - Work It Daily | Where Careers Go To Grow   https://www.workitdaily.com/how-to-quit-your-job
6604 :  WORKANA > Find talent, find work   https://www.workana.com/job/insight/web-design-criacao-de-banners-e-artes-para-midias?ref=project_view
6605 :  Frontend Angular Developer Job in Jaipur  - Advaita Services - 2 to 3 Years of experience - Worknrby   https://www.worknrby.com/job-opportunity-frontend-angular-developer-advaita-services-civil-lines-jaipur-rajasthan-302007-india-rajasthan-633165
6612 :  
Draw Your Success Moment to Storyboard Your Next Career-Building Step | shelleyupton   https://shelleyupton.wordpress.com/2013/11/11/draw-your-success-moment-to-storyboard-your-next-career-building-step/
6615 :  The air-hostess job. – Literature Is My Porn   https://literatureismyporn.wordpress.com/2015/09/23/the-air-hostess-job/?like_comment=1020&_wpnonce=9ff03d4e43
6617 :  4 Truths 

6739 :  Full Time Work From Home Jobs: 20 Companies Hiring NOW! – Work At Home Group   https://workathomegroup.com/full-time-work-from-home-jobs-20-companies-hiring-now/
6741 :  Not Found - WorkForThem   https://workforthem.com/job/24492/editorial-designer-amsterdam-at-the-correspondent
6742 :  Head of Marketing - Scale up job at BlueOptima,   January 2021   https://workinstartups.com/job-board/job/100370/head-of-marketing-scale-up-at-blueoptima/
6743 :  WORKANA > Find talent, find work   https://www.workana.com/job/insight/diseno-pagina-web-venta-de-productos-para-la-salud-que-sea-dinamica-y-me-den-opcion-para-cambiar-informacion-constantemente-acep?ref=project_view
6745 :  WorkSourceWA - occda   https://seeker.worksourcewa.com/jobsearch/PowerSearch.aspx?jt=6&tm=14&q=occda&where=Washington&rad=10&sort=rv.di.dt%2Cdt.rv.di&rad_units=miles&pp=25&vw=b&setype=2&jbf67816=88993&brd=3%2C1
6746 :  Innovating the Beauty Industry with Rica Elysee –                       La Femme Exec   https://l

6846 :  Online Sales Executive Job in Jaipur  - Forever Star India Awards - 0 to 1 Year of experience - Worknrby   https://www.worknrby.com/job-opportunity-nirman-nagar-ABFFC4
6847 :  
career | Kalí's Musings   https://kalipr.wordpress.com/tag/career/
6851 :  Wordpress Job in Jaipur  - Webmaster Driver - 2 to 3 Years of experience - Worknrby   https://www.worknrby.com/job-opportunity-wordpress-webmaster-driver-surya-nagar-gopal-pura-mode-jaipur-rajasthan-india-rajasthan-2F5BC0
6854 :  Narrative | WorksHub | Ai Works   https://ai.works-hub.com/companies/narrative/jobs
6855 :  How to find a remote job and work remotely - Workew Learn   https://workew.com/learn/courses/find-remote-job-work-remotely/
6856 :  Workopolis | Workopolis   https://www.workopolis.com/jobsearch/viewjob/bhl2RjgfkGc3FtxSKV0tfAYiCjRKa7uaN4HwaSY5GHhP2ywwCbk8IA?ak=online+teaching&l=edmonton+ab
6857 :  jobs – I Tripped Over a Stone   https://itrippedoverastone.wordpress.com/tag/jobs/
6859 :  Jewellery Designer Job in Ja

6970 :  Writing Is A Great Second Career After Working In GIS - Work It Daily | Where Careers Go To Grow   https://www.workitdaily.com/writing-second-career-gis
6973 :  Terraform Jobs - January 2021 | Functional Works | Functional Works   https://functional.works-hub.com/terraform-jobs
6976 :  WorkPac Group Boilermaker · WorkPac Group   https://www.workpacgroup.com/job/boilermaker-115
6977 :  How to know I’m a woman – Henna Sjöblom – Whisper and the Roar   https://whisperandtheroar.wordpress.com/2018/03/07/how-to-know-im-a-woman-henna-sjoblom/
6978 :  3 Ways To Boost Your Chances Of Landing A Job - Work It Daily | Where Careers Go To Grow   https://www.workitdaily.com/job-land-boost-chances
6979 :  Yet Again, Big Government Interferes With Growth And Jobs - Workforce Fairness Institute   https://workforcefairness.com/yet-big-government-interferes-growth-jobs/
6980 :  Game Developmentの仕事依頼 - 料金・実積 クラウドソーシング【ワークシフト】   https://workshift-sol.com/jobs/search?smallcategory_id%5B%5D=15&all=0
